# 

# Test

In [12]:
import pandas as pd
from apyori import apriori

In [13]:
data = pd.read_csv('data/D1.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          131706 non-null  object 
 1   Customer_ID   131706 non-null  int64  
 2   Sales_ID      131706 non-null  int64  
 3   SKU_Category  131706 non-null  object 
 4   SKU           131706 non-null  object 
 5   Quantity      131706 non-null  float64
 6   Sales_Amount  131706 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.0+ MB


In [14]:
# convert all columns to lower case
data.columns = data.columns.str.lower()

In [104]:
data

,date,customer_id,sales_id,sku_category,sku,quantity,sales_amount
0,2016-01-02,2547,1,X52,0EM7L,1.0,3.13
1,2016-01-02,822,2,2ML,68BRQ,1.0,5.46
2,2016-01-02,3686,3,0H2,CZUZX,1.0,6.35
3,2016-01-02,3719,4,0H2,549KK,1.0,5.59
4,2016-01-02,9200,5,0H2,K8EHH,1.0,6.88
...,...,...,...,...,...,...,...
131701,2016-07-04,20203,32900,IEV,FO112,3.0,6.46
131702,2016-07-04,20203,32900,N8U,I36F2,1.0,4.50
131703,2016-07-04,20203,32900,U5F,4X8P4,1.0,5.19
131704,2016-07-04,20203,32900,0H2,ZVTO4,1.0,4.57


In [16]:
# convert Date to a date object
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y')

In [6]:
# Convert Sales_ID, and Customer_ID to a string type
data['sales_id'] = data['sales_id'].astype(str)
data['customer_id'] = data['customer_id'].astype(str)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          131706 non-null  datetime64[ns]
 1   customer_id   131706 non-null  int64         
 2   sales_id      131706 non-null  int64         
 3   sku_category  131706 non-null  object        
 4   sku           131706 non-null  object        
 5   quantity      131706 non-null  float64       
 6   sales_amount  131706 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 7.0+ MB


In [20]:
# Define the Transactional Data
transactions = data.groupby('sales_id')['sku_category'].apply(list)
transactions_list = list(transactions)

In [23]:
data.groupby(['customer_id','date'])['sales_id'].count()

customer_id  date      
1            2016-01-22    2
2            2016-03-24    1
             2016-06-19    1
3            2016-01-02    3
4            2016-07-11    1
                          ..
22624        2016-12-02    2
22625        2016-12-02    4
             2016-12-14    3
             2016-12-19    1
             2016-12-31    1
Name: sales_id, Length: 62727, dtype: int64

In [25]:
1 / len(data['sku_category'].value_counts())

0.0053475935828877

In [203]:
results = list(apriori(transactions_list, min_support=0.005, min_confidence=0.2))

In [204]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = Left side of rules, items_add = Right side
            # support, confidence and lift for respoective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                          rule_set.support, rule.confidence, rule.lift])
            
            # Typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_rule', 'Right_rule', 'Support', 'Confidence', 'Lift'])


In [205]:
results_df = convert_apriori_results_to_pandas_df(results)

In [206]:
results_df.sort_values(by='Lift', ascending=False)

,Left_rule,Right_rule,Support,Confidence,Lift
77,"N8U,LPF",OXH,0.006973,0.413761,9.919540
74,"IEV,N8U",OXH,0.006076,0.396569,9.507370
15,9ZX,FU5,0.007220,0.380293,9.100304
64,"OXH,LPF",FU5,0.007313,0.364407,8.720148
79,"N8U,OXH",LPF,0.006973,0.626389,8.715011
...,...,...,...,...,...
45,OXH,N8U,0.011131,0.266864,1.742160
28,FU5,N8U,0.010173,0.243433,1.589195
33,IEV,N8U,0.015321,0.240476,1.569887
39,LPF,N8U,0.016852,0.234459,1.530609


In [211]:
# filter results_df for rules with 01F in the left_rule or the right rule
filtered_results = results_df[results_df['Left_rule'].str.contains('01F') | results_df['Right_rule'].str.contains('01F')]

In [218]:
filtered_results.sort_values(by='Support', ascending=False)

,Left_rule,Right_rule,Support,Confidence,Lift
3,01F,IEV,0.012909,0.481268,7.553841
4,IEV,01F,0.012909,0.202621,7.553841
5,01F,LPF,0.012198,0.454755,6.327052
7,01F,OXH,0.008163,0.304323,7.295851
2,01F,FU5,0.007498,0.279539,6.689284
6,01F,N8U,0.007421,0.276657,1.806089
48,01F,"IEV,LPF",0.006447,0.240346,7.715161
49,"IEV,01F",LPF,0.006447,0.499401,6.948219
50,"LPF,01F",IEV,0.006447,0.528517,8.295449
51,"IEV,LPF",01F,0.006447,0.206948,7.715161


In [213]:
0.013*100

1.3

In [50]:
data = data.sort_values(by=['Customer_ID', 'Date'])

KeyError: 'Customer_ID'

In [14]:
data.head()

,Date,Customer_ID,Sales_ID,SKU_Category,SKU,Quantity,Sales_Amount
7284,2016-01-22,1,3442,0H2,6OUVC,1.0,10.50
7285,2016-01-22,1,3442,N8U,CEBU8,1.0,5.79
14355,2016-02-12,10,6893,SJS,UTE9R,1.0,110.31
312,2016-01-03,100,181,FEW,3QNEE,1.0,8.32
313,2016-01-03,100,181,MU3,O9LF5,1.0,4.81


In [15]:
transactions= data.groupby('Customer_ID')['SKU_Category'].apply(list)

In [16]:
# transactions2 = transactions['SKU_Category'].apply(list)

In [17]:
sequences = transactions.values.tolist()

In [18]:
sequences[0:5]

[['0H2', 'N8U'], ['SJS'], ['FEW', 'MU3'], ['LSD'], ['1VL', 'J4R', 'P42']]

In [19]:
from collections import defaultdict
import subprocess
import re
    

In [20]:

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 
                     'seq_rule_input.txt', 'seq_rule_output.txt', 
                     supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [21]:
get_association_rules(sequences, 0.01, 0.5)

,Left_rule,Right_rule,Support,Confidence
0,"[N8U, OXH]",[LPF],0.021790,0.542354
1,"[N8U, OXH, IEV]",[LPF],0.014453,0.560892
2,"[N8U, OXH, FU5]",[LPF],0.010166,0.580808
3,"[N8U, IEV, FU5]",[LPF],0.010564,0.537079
4,"[N8U, FU5]",[LPF],0.017282,0.506477
5,"[N8U, 01F]",[IEV],0.011536,0.509766
6,[OXH],[LPF],0.039425,0.522248
7,"[OXH, U5F]",[LPF],0.011624,0.513672
8,"[OXH, 01F]",[LPF],0.010696,0.564103
9,"[OXH, IEV]",[LPF],0.022055,0.519251
